Use continuous geographic, hydrological, climate, and meteorological datasets as input to train neural network(NN) and random forest(RF) models, to determine the major environmentl drivers of algae growth in lakes.
Use Google Cloud Platform (GCP) to extract relative data from its Earth Engine Dataset and run further NN models

In [1]:
import ee
service_account = '429406912917-compute@developer.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'credentials.json')
ee.Initialize(credentials)

In [2]:
### extract climate data from ERA 5
sensor='ECMWF/ERA5/DAILY'
start_date,end_date='2016-09-01','2022-10-01'
aoi =ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77)

era5 = ee.ImageCollection(sensor) \
       .filterBounds(aoi)  \
       .filterDate(ee.Date(start_date), ee.Date(end_date))
era5_2mt = era5.select('mean_2m_air_temperature')

In [5]:
##preview some imagery
img=era5_2mt.first()
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1578495016371452,
 'id': 'ECMWF/ERA5/DAILY/20160901',
 'properties': {'system:time_start': 1472688000000,
  'month': 9,
  'year': 2016,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1472774400000,
  'system:asset_size': 36675608,
  'day': 1,
  'system:index': '20160901'}}

In [6]:
## export to imagery to Google Cloud Platform (GCP) buckets 
export_config = {
    'image': img,
    'region': aoi,
    'crs': 'EPSG:4326',
    'scale': 100000,    
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'ERA5_2mt',
    'bucket':   'AlgaeDrive',  
    'maxPixels':1e10,
    'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature'
}
task=ee.batch.Export.image.toCloudStorage(**export_config)
#task=ee.batch.Export.image.toDrive(**export_config)
task.start()
import time
time.sleep(10)
print(task.status())

{'state': 'READY', 'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature', 'creation_timestamp_ms': 1680028182464, 'update_timestamp_ms': 1680028182464, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UQFQZPCG5XE5SKJFP2QS4TKZ', 'name': 'projects/earthengine-legacy/operations/UQFQZPCG5XE5SKJFP2QS4TKZ'}


In [ ]:
##iterate all the image to export
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(nImages):
	img=ee.Image(img_list.get(i)).clip(aoi)
	product_id=img.get('PRODUCT_ID').getInfo()
	export_config = {
		'image': img.float(),
		'region': aoi_clip,
		'crs': f'EPSG:{crs_epsg}',
		'scale': 300,    #default in 300 pixels of OLCI
		'fileFormat': 'GeoTIFF',
		'fileNamePrefix': f'mali_test/{product_id}',
		'bucket':   f'mali_test',  
		'maxPixels':1e10,
		'description': 'test'
	}

In [36]:
img = era5_2mt.first().clip(aoi)
#print(img.get('bands').getInfo())
#img.get('id').getInfo()
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [9, 5],
   'origin': [337, 160],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1578495016371452,
 'id': 'ECMWF/ERA5/DAILY/20160901',
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[-95.4, 48.88],
     [-93.75, 48.88],
     [-93.75, 49.77],
     [-95.4, 49.77],
     [-95.4, 48.88]]]},
  'system:time_start': 1472688000000,
  'month': 9,
  'year': 2016,
  'system:time_end': 1472774400000,
  'system:asset_size': 36675608,
  'day': 1,
  'system:index': '20160901'}}

In [19]:
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(20):
  img = ee.Image(img_list.get(i))
  #img=ee.Image(era5_2mt.get(0)).clip(aoi)
  print(img.get('PRODUCT_ID').getInfo())

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
